In [1]:
import torch
from torch import nn
from model import *

In [2]:
dim = 288
n_layers = 6
n_heads = 6
multiple_of = 32
dropout = 0.0

batch_size = 2  # if gradient_accumulation_steps > 1, this is the micro-batch size
max_seq_len = 256


model_args = dict(
    dim=dim,
    n_layers=n_layers,
    n_heads=n_heads,
    n_kv_heads=n_heads,
    vocab_size=32000,
    multiple_of=multiple_of,
    max_seq_len=max_seq_len,
    #dropout=dropout,
    add_zero_attn=False,
    softmax="softmax1",
    flash=False,

)

# model_args=dict(dim=768, n_layers=12, n_heads=12, 
#                 n_kv_heads=12, vocab_size=32000, multiple_of=32, 
#                 norm_eps=1e-05, max_seq_len=1024, dropout=0.0, 
#                 add_zero_attn=False, softmax='softmax1', flash=False)

model_args = ModelArgs(**model_args)

In [3]:
model = Transformer(model_args)

## Hooks

In [4]:
def hook_fn(module, input, output):
    "Hook function to store outputs in a .output attribute"
    if hasattr(module, "max_att"):
        module.max_att = max(output.max(), module.max_att)
    else:
        module.max_att = output.max()

In [5]:

# Register with all the blocks' attention modules
i = 0
for b in model.layers:
    if hasattr(b, 'attention'):
        # print("Registering forward hook for %s" % str(b))
        b.attention.register_forward_hook(hook_fn)
        i+=1

In [6]:
x = torch.randint(0, model_args.vocab_size, (batch_size, model_args.max_seq_len) )
out = model(x)

tensor(inf, grad_fn=<MaxBackward1>) tensor(7.5356e-15, grad_fn=<MinBackward1>)
tensor(inf, grad_fn=<MaxBackward1>) tensor(1.4565, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_fn=<MinBackward1>)
tensor(nan, grad_fn=<MaxBackward1>) tensor(nan, grad_f

In [7]:
activations[0].shape

NameError: name 'activations' is not defined

In [ ]:
model.layers[0].attention.max_activation

tensor(0.1177, grad_fn=<MaxBackward1>)

In [ ]:
import torch

m = torch.nn.Linear(2,1)

m.register_forward_hook(lambda m,i,o: print("I am capatin Hook"))

# works as expexted
m(torch.randn(2))

# does not work
m.forward(torch.randn(2))


I am capatin Hook


tensor([-1.7607], grad_fn=<AddBackward0>)

## add_zero_attn

In [4]:
x = torch.randint(0, model_args.vocab_size, (batch_size, model_args.max_seq_len) )
out = model(x)

TypeError: TransformerBlock.forward() takes 3 positional arguments but 4 were given

In [11]:
out

(tensor([[[-0.3148,  0.2506,  0.3788,  ..., -0.1718, -0.0094,  0.1246]],
 
         [[ 0.2734, -0.1760, -0.1170,  ..., -0.5049, -0.0271,  0.5356]]],
        grad_fn=<UnsafeViewBackward0>),
 None)

In [ ]:
F.multi_head_attention_forward??

Signature:
F.multi_head_attention_forward(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    embed_dim_to_check: int,
    num_heads: int,
    in_proj_weight: Optional[torch.Tensor],
    in_proj_bias: Optional[torch.Tensor],
    bias_k: Optional[torch.Tensor],
    bias_v: Optional[torch.Tensor],
    add_zero_attn: bool,
    dropout_p: float,
    out_proj_weight: torch.Tensor,
    out_proj_bias: Optional[torch.Tensor],
    training: bool = True,
    key_padding_mask: Optional[torch.Tensor] = None,
    need_weights: bool = True,
    attn_mask: Optional[torch.Tensor] = None,
    use_separate_proj_weight: bool = False,
    q_proj_weight: Optional[torch.Tensor] = None,
    k_proj_weight: Optional[torch.Tensor] = None,
    v_proj_weight: Optional[torch.Tensor] = None,
    static_k: Optional[torch.Tensor] = None,
    static_v: Optional[torch.Tensor] = None,
    average_attn_weights: bool = True,
    is_causal: bool = False,
) -> Tuple[torch.Tensor, Optional[torch.Ten